# 12.1 Decorator là gì

Decorator là những functions thay đổi tính năng của một function, method hay class một cách dynamic, mà không phải sử dụng lớp con. Nó rất tiện lợi khi bạn muốn mở rộng tính năng của một function mà bạn không muốn thay đổi nó.

# 12.2 Function Decorator
**Cú pháp:** Phải có dấu @ trước hàm cần mở rộng tính năng.
```python
@decorator 
def func(arg):
    ...
func(99)
```
Sẽ thay thế cho các dòng lệnh sau đây:
```python
def func(arg):
    ...
func=decorator(func)  # decorator(func)(99)
func(99)
```


In [10]:
def decorator(F):
    return F
@decorator
def func():
    print('x')
func()

x


# 12.2 Class Decorator

In [15]:
def decorator(cls): 
    class Wrapper:
        def __init__(self, *args):   # On instance creation
            self.wrapped = cls(*args)
        def __getattr__(self, name): # On attribute fetch
            return getattr(self.wrapped, name)
    return Wrapper

@decorator
class C: 
    def __init__(self, x, y): # Run by Wrapper.__init__
        self.attr = 'spam'
        
x = C(6, 7)
print(x.attr)

spam


## Nested decorator

In [16]:
def d1(F): return F
def d2(F): return F
def d3(F): return F
@d1
@d2
@d3
def func():        # func = d1(d2(d3(func)))
    print('spam')
func()

spam


In [17]:
def d1(F): return lambda: 'X' + F()
def d2(F): return lambda: 'Y' + F()
def d3(F): return lambda: 'Z' + F()
@d1
@d2
@d3
def func():          # func = d1(d2(d3(func)))
    return 'spam'
print(func()) 

XYZspam


# 12.3 Argument decorator

In [19]:
class tracer:
    def __init__(self, func): # On @ decoration: save original func
        self.calls = 0
        self.func = func
    def __call__(self, *args): # On later calls: run original func
        self.calls += 1
        print('call %s to %s' % (self.calls, self.func.__name__))
        self.func(*args)
        
@tracer
def spam(a, b, c): # spam = tracer(spam)
    print(a + b + c)
    
spam(1,2,3)

call 1 to spam
6


In [20]:
spam('a', 'b', 'c')

call 2 to spam
abc


In [21]:
spam.calls

2

In [22]:
spam

In [23]:
calls = 0
def tracer(func, *args):
    global calls
    calls += 1
    print('call %s to %s' % (calls, func.__name__))
    func(*args)
    
def spam(a, b, c):
    print(a, b, c)

spam(1, 2, 3)

1 2 3


In [15]:
tracer(spam, 1, 2, 3)

call 1 to spam
1 2 3


In [24]:
# Class instance attribute
class tracer: 
    def __init__(self, func): 
        self.calls = 0                    # Save func for later call
        self.func = func
    def __call__(self, *args, **kwargs): # On call to original function
        self.calls += 1
        print('call %s to %s' % (self.calls, self.func.__name__))
        return self.func(*args, **kwargs)
    
@tracer
def spam(a, b, c): # Same as: spam = tracer(spam)
    print(a + b + c) # Triggers tracer.__init__
    
@tracer
def eggs(x, y): # Same as: eggs = tracer(eggs)
    print(x ** y) # Wraps eggs in a tracer object
    
spam(1, 2, 3)      # Really calls tracer instance: runs tracer.__call__

spam(a=4, b=5, c=6) # spam is an instance attribute

eggs(2, 16) # Really calls tracer instance, self.func is eggs

eggs(4, y=4) # self.calls is per-decoration here

call 1 to spam
6
call 2 to spam
15
call 1 to eggs
65536
call 2 to eggs
256


In [17]:
# Enclosing scope and gobal
calls = 0
def tracer(func): # State via enclosing scope and global
    def wrapper(*args, **kwargs): # Instead of class attributes
        global calls # calls is global, not per-function
        calls += 1
        print('call %s to %s' % (calls, func.__name__))
        return func(*args, **kwargs)
    return wrapper

@tracer
def spam(a, b, c): # Same as: spam = tracer(spam)
    print(a + b + c)
    
@tracer
def eggs(x, y): # Same as: eggs = tracer(eggs)
    print(x ** y)
    
spam(1, 2, 3) # Really calls wrapper, assigned to spam
spam(a=4, b=5, c=6) # wrapper calls spam

eggs(2, 16) # Really calls wrapper, assigned to eggs
eggs(4, y=4)

call 1 to spam
6
call 2 to spam
15
call 3 to eggs
65536
call 4 to eggs
256


In [18]:
# Enclosing scope and nonlocal
def tracer(func): # State via enclosing scope and nonlocal
    calls = 0 # Instead of class attrs or global
    def wrapper(*args, **kwargs): # calls is per-function, not global
        nonlocal calls
        calls += 1
        print('call %s to %s' % (calls, func.__name__))
        return func(*args, **kwargs)
    return wrapper

@tracer
def spam(a, b, c): # Same as: spam = tracer(spam)
    print(a + b + c)
    
@tracer
def eggs(x, y): # Same as: eggs = tracer(eggs)
    print(x ** y)
    
spam(1, 2, 3) # Really calls wrapper, bound to func
spam(a=4, b=5, c=6) # wrapper calls spam
eggs(2, 16) # Really calls wrapper, bound to eggs
eggs(4, y=4) # Nonlocal calls _is_ per-decoration here

call 1 to spam
6
call 2 to spam
15
call 1 to eggs
65536
call 2 to eggs
256


In [19]:
# Function attribute
def tracer(func): # State via enclosing scope and func attr
    def wrapper(*args, **kwargs): # calls is per-function, not global
        wrapper.calls += 1
        print('call %s to %s' % (wrapper.calls, func.__name__))
        return func(*args, **kwargs)
    wrapper.calls = 0
    return wrapper

@tracer
def spam(a, b, c): # Same as: spam = tracer(spam)
    print(a + b + c)
    
@tracer
def eggs(x, y): # Same as: eggs = tracer(eggs)
    print(x ** y)
    
spam(1, 2, 3) # Really calls wrapper, assigned to spam
spam(a=4, b=5, c=6) # wrapper calls spam
eggs(2, 16) # Really calls wrapper, assigned to eggs
eggs(4, y=4) # wrapper.calls _is_ per-decoration here

call 1 to spam
6
call 2 to spam
15
call 1 to eggs
65536
call 2 to eggs
256


# 12.4 Sai lầm khi sử dụng Decorator

In [26]:
class tracer:
    def __init__(self, func): # On @ decorator
        self.calls = 0 # Save func for later call
        self.func = func
    def __call__(self, *args, **kwargs): # On call to original function
        self.calls += 1
        print('call %s to %s' % (self.calls, self.func.__name__))
        return self.func(*args, **kwargs)

@tracer
def spam(a, b, c): # spam = tracer(spam)
    print(a + b + c)

spam(1, 2, 3)
spam(a=4, b=5, c=6)

call 1 to spam
6
call 2 to spam
15


In [27]:
class Person:
    def __init__(self, name, pay):
        self.name = name
        self.pay = pay
        
    @tracer
    def giveRaise(self, percent): # giveRaise = tracer(giveRaise)
        self.pay *= (1.0 + percent)
        
    @tracer
    def lastName(self): # lastName = tracer(lastName)
        return self.name.split()[-1]

```python
bob = Person('Bob Smith', 50000) # tracer remembers method funcs
bob.giveRaise(.25)
```
**Output:**
```python
call 1 to giveRaise
----------------------------------------------------------------
TypeError                      Traceback (most recent call last)
<ipython-input-25-788199bebc0b> in <module>()
      1 bob = Person('Bob Smith', 50000) # tracer remembers method funcs
----> 2 bob.giveRaise(.25)

<ipython-input-20-e04dfdc1d359> in __call__(self, *args, **kwargs)
      6         self.calls += 1
      7         print('call %s to %s' % (self.calls, self.func.__name__))
----> 8         return self.func(*args, **kwargs)

TypeError: giveRaise() missing 1 required positional argument: 'percent'
```

In [30]:
def tracer(func): # Use function, not class with __call__
    calls = 0 # Else "self" is decorator instance only!
    def onCall(*args, **kwargs): # Or in 2.X+3.X: use [onCall.calls += 1]
        nonlocal calls
        calls += 1
        print('call %s to %s' % (calls, func.__name__))
        return func(*args, **kwargs)
    return onCall

if __name__ == '__main__':
# Applies to simple functions
    @tracer
    def spam(a, b, c): # spam = tracer(spam)
        print(a + b + c) # onCall remembers spam
        
    @tracer
    def eggs(N):
        return 2 ** N
    spam(1, 2, 3) # Runs onCall(1, 2, 3)
    spam(a=4, b=5, c=6)
    print(eggs(32))
    
    class Person:
        def __init__(self, name, pay):
            self.name = name
            self.pay = pay
            
        @tracer
        def giveRaise(self, percent): # giveRaise = tracer(giveRaise)
            self.pay *= (1.0 + percent) # onCall remembers giveRaise
            
        @tracer
        def lastName(self): # lastName = tracer(lastName)
            return self.name.split()[-1]
        
    print('methods...')
    bob = Person('Bob Smith', 50000)
    sue = Person('Sue Jones', 100000)
    print(bob.name, sue.name)
    sue.giveRaise(.10) # Runs onCall(sue, .10)
    print(int(sue.pay))
    print(bob.lastName(), sue.lastName())

call 1 to spam
6
call 2 to spam
15
call 1 to eggs
4294967296
methods...
Bob Smith Sue Jones
call 1 to giveRaise
110000
call 1 to lastName
call 2 to lastName
Smith Jones


## Timing call

In [34]:
import time, sys
force = list if sys.version_info[0] == 3 else (lambda X: X)

class timer:
    def __init__(self, func):
        self.func = func
        self.alltime = 0
    def __call__(self, *args, **kargs):
        start = time.perf_counter()
        result = self.func(*args, **kargs)
        elapsed = time.perf_counter() - start
        self.alltime += elapsed
        print('%s: %.5f, %.5f' % (self.func.__name__, elapsed, self.alltime))
        return result
    
@timer
def listcomp(N):
    return [x * 2 for x in range(N)]

@timer
def mapcall(N):
    return force(map((lambda x: x * 2), range(N)))

result = listcomp(5) # Time for this call, all calls, return value
listcomp(50000)
listcomp(500000)
listcomp(1000000)
print(result)
print('allTime = %s' % listcomp.alltime) # Total time for all listcomp calls

print('')
result = mapcall(5)
mapcall(50000)
mapcall(500000)
mapcall(1000000)
print(result)
print('allTime = %s' % mapcall.alltime) # Total time for all mapcall calls
print('\n**map/comp = %s' % round(mapcall.alltime / listcomp.alltime, 3))

listcomp: 0.00001, 0.00001
listcomp: 0.00847, 0.00848
listcomp: 0.10537, 0.11385
listcomp: 0.20245, 0.31630
[0, 2, 4, 6, 8]
allTime = 0.3163037799995436

mapcall: 0.00001, 0.00001
mapcall: 0.01281, 0.01282
mapcall: 0.14939, 0.16221
mapcall: 0.30407, 0.46628
[0, 2, 4, 6, 8]
allTime = 0.4662753780003186

**map/comp = 1.474


In [37]:
import timeit
timeit.timeit(number=1, stmt=lambda: listcomp(1000000))

0.23622975000012048

In [37]:
# Add Decorator Argument
import time

def timer(label='', trace=True): # On decorator args: retain args
    class Timer:
        def __init__(self, func): # On @: retain decorated func
            self.func = func
            self.alltime = 0
        def __call__(self, *args, **kargs): # On calls: call original
            start = time.perf_counter()
            result = self.func(*args, **kargs)
            elapsed = time.perf_counter() - start
            self.alltime += elapsed
            if trace:
                format = '%s %s: %.5f, %.5f'
                values = (label, self.func.__name__, elapsed, self.alltime)
                print(format % values)
            return result
    return Timer

import sys
force = list if sys.version_info[0] == 3 else (lambda X: X)

@timer(label='[CCC]==>')
def listcomp(N):        # Like listcomp = timer(...)(listcomp)
    return [x * 2 for x in range(N)]

@timer(trace=True, label='[MMM]==>')
def mapcall(N):
    return force(map((lambda x: x * 2), range(N)))

for func in (listcomp, mapcall):
    result = func(5)   # Time for this call, all calls, return value
    func(50000)
    func(500000)
    func(1000000)
    print(result)
    print('allTime = %s\n' % func.alltime) # Total time for all calls
print('**map/comp = %s' % round(mapcall.alltime / listcomp.alltime, 3))

[CCC]==> listcomp: 0.00001, 0.00001
[CCC]==> listcomp: 0.00757, 0.00758
[CCC]==> listcomp: 0.10688, 0.11446
[CCC]==> listcomp: 0.19828, 0.31274
[0, 2, 4, 6, 8]
allTime = 0.31274153100093827

[MMM]==> mapcall: 0.00001, 0.00001
[MMM]==> mapcall: 0.01250, 0.01251
[MMM]==> mapcall: 0.15113, 0.16364
[MMM]==> mapcall: 0.30536, 0.46900
[0, 2, 4, 6, 8]
allTime = 0.4689955550011291

**map/comp = 1.5


In [39]:
@timer(trace=False) # No tracing, collect total time
def listcomp(N):
    return [x * 2 for x in range(N)]
x = listcomp(5000)
listcomp.alltime

0.0006686269998681382

In [42]:
listcomp

<__main__.timer.<locals>.Timer at 0x3868ab0>

In [44]:
listcomp.alltime

0.002036711999835461

# 12.5 Pattern Singleton

In [40]:
instances = {}
def singleton(aClass): # On @ decoration
    def onCall(*args, **kwargs): # On instance creation
        if aClass not in instances: # One dict entry per class
            instances[aClass] = aClass(*args, **kwargs)
        return instances[aClass]
    return onCall

@singleton # Person = singleton(Person)
class Person: # Rebinds Person to onCall
    def __init__(self, name, hours, rate): # onCall remembers Person
        self.name = name
        self.hours = hours
        self.rate = rate
    def pay(self):
        return self.hours * self.rate
    
@singleton # Spam = singleton(Spam)
class Spam: # Rebinds Spam to onCall
    def __init__(self, val): # onCall remembers Spam
        self.attr = val
        
bob = Person('Bob', 40, 10) # Really calls onCall
print(bob.name, bob.pay())

sue = Person('Sue', 50, 20) # Same, single object
print(sue.name, sue.pay())

X = Spam(val=42) # One Person, one Spam
Y = Spam(99)
print(X.attr, Y.attr)

Bob 400
Bob 400
42 42


In [47]:
# nonlocal
def singleton(aClass): # On @ decoration
    instance = None
    def onCall(*args, **kwargs): # On instance creation
        nonlocal instance # 3.X and later nonlocal
        if instance == None:
            instance = aClass(*args, **kwargs) # One scope per class
        return instance
    return onCall

In [41]:
# function attribute
def singleton(aClass): # On @ decoration
    def onCall(*args, **kwargs): # On instance creation
        if onCall.instance == None:
            onCall.instance = aClass(*args, **kwargs) # One function per class
        return onCall.instance
    onCall.instance = None
    return onCall

class singleton:
    def __init__(self, aClass): # On @ decoration
        self.aClass = aClass
        self.instance = None
    def __call__(self, *args, **kwargs): # On instance creation
        if self.instance == None:
            self.instance = self.aClass(*args, **kwargs) # One instance per class
        return self.instance

In [43]:
class Wrapper:
    def __init__(self, object):
        self.wrapped = object # Save object
    def __getattr__(self, attrname):
        print('Trace:', attrname) # Trace fetch
        return getattr(self.wrapped, attrname) # Delegate fetch

In [44]:
x = Wrapper([1,2,3])
x.append(4)

Trace: append


In [45]:
x.wrapped

[1, 2, 3, 4]

In [46]:
x = Wrapper({"a": 1, "b": 2}) # Wrap a dictionary
list(x.keys())

Trace: keys


['a', 'b']

In [47]:
def Tracer(aClass): # On @ decorator
    class Wrapper:
        def __init__(self, *args, **kargs): # On instance creation
            self.fetches = 0
            self.wrapped = aClass(*args, **kargs) # Use enclosing scope name
        def __getattr__(self, attrname):
            print('Trace: ' + attrname) # Catches all but own attrs
            self.fetches += 1
            return getattr(self.wrapped, attrname) # Delegate to wrapped obj
    return Wrapper

if __name__ == '__main__':
    
    @Tracer
    class Spam: # Spam = Tracer(Spam)
        def display(self): # Spam is rebound to Wrapper
            print('Spam!' * 8)
            
    @Tracer
    class Person: # Person = Tracer(Person)
        def __init__(self, name, hours, rate): # Wrapper remembers Person
            self.name = name
            self.hours = hours
            self.rate = rate
        def pay(self): # Accesses outside class traced
            return self.hours * self.rate # In-method accesses not traced
        
food = Spam() # Triggers Wrapper()
food.display() # Triggers __getattr__
print([food.fetches])

bob = Person('Bob', 40, 50) # bob is really a Wrapper
print(bob.name) # Wrapper embeds a Person
print(bob.pay())

print('')
sue = Person('Sue', rate=100, hours=60) # sue is a different Wrapper
print(sue.name) # with a different Person
print(sue.pay())

print(bob.name) # bob has different state
print(bob.pay())
print([bob.fetches, sue.fetches])

Trace: display
Spam!Spam!Spam!Spam!Spam!Spam!Spam!Spam!
[1]
Trace: name
Bob
Trace: pay
2000

Trace: name
Sue
Trace: pay
6000
Trace: name
Bob
Trace: pay
2000
[4, 2]


In [56]:
@Tracer
class MyList(list): pass
x = MyList([1, 2, 3])
x.append(4)

Trace: append


In [57]:
 x.wrapped

[1, 2, 3, 4]

In [58]:
WrapList = Tracer(list) # Or perform decoration manually
x = WrapList([4, 5, 6]) # Else subclass statement required
x.append(7)

Trace: append


In [59]:
x.wrapped

[4, 5, 6, 7]

In [60]:
# Retaining multiple instance
class Tracer:
    def __init__(self, aClass): # On @decorator
        self.aClass = aClass # Use instance attribute
    def __call__(self, *args): # On instance creation
        self.wrapped = self.aClass(*args) # ONE (LAST) INSTANCE PER CLASS!
        return self
    def __getattr__(self, attrname):
        print('Trace: ' + attrname)
        return getattr(self.wrapped, attrname)
    
@Tracer # Triggers __init__
class Spam: # Like: Spam = Tracer(Spam)
    def display(self):
        print('Spam!' * 8)

In [61]:
@Tracer
class Person: # Person = Tracer(Person)
    def __init__(self, name): # Wrapper bound to Person
        self.name = name
        
bob = Person('Bob') # bob is really a Wrapper
print(bob.name) # Wrapper embeds a Person
Sue = Person('Sue')
print(sue.name) # sue overwrites bob
print(bob.name)

Trace: name
Bob
Trace: name
Sue
Trace: name
Sue


In [50]:
# Manager function
instances = {}
def getInstance(aClass, *args, **kwargs):
    if aClass not in instances:
        instances[aClass] = aClass(*args, **kwargs)
    return instances[aClass]

bob = getInstance(Person, 'Bob', 40, 10) 

In [51]:
instances = {}
def getInstance(object):
    aClass = object.__class__
    if aClass not in instances:
        instances[aClass] = object
    return instances[aClass]

bob = getInstance(Person('Bob', 40, 10))

In [64]:
# Managing function
registry = {}
def register(obj): # Both class and func decorator
    registry[obj.__name__] = obj # Add to registry
    return obj # Return obj itself, not a wrapper

@register
def spam(x):
    return(x ** 2) # spam = register(spam)

@register
def ham(x):
    return(x ** 3)

@register
class Eggs: # Eggs = register(Eggs)
    def __init__(self, x):
        self.data = x ** 4
        
def __str__(self):
    return str(self.data)

print('Registry:')
for name in registry:
    print(name, '=>', registry[name], type(registry[name]))
    
print('\nManual calls:')
print(spam(2)) # Invoke objects manually
print(ham(2)) # Later calls not intercepted
X = Eggs(2)
print(X)

print('\nRegistry calls:')
for name in registry:
    print(name, '=>', registry[name](2)) # Invoke from registry

Registry:
spam => <function spam at 0x0544EA50> <class 'function'>
ham => <function ham at 0x03819B70> <class 'function'>
Eggs => <class '__main__.Eggs'> <class 'type'>

Manual calls:
4
8

Registry calls:
spam => 4
ham => 8
Eggs => <__main__.Eggs object at 0x05464AF0>


In [52]:
def decorate(func):
    func.marked = True # Assign function attribute for later use
    return func

@decorate
def spam(a, b):
    return a + b
spam.marked

True

In [54]:
def annotate(text): # Same, but value is decorator argument
    def decorate(func):
        func.label = text
        return func
    return decorate

@annotate('spam data')
def spam(a, b): # spam = annotate(...)(spam)
    return a + b

spam(1, 2), spam.label

(3, 'spam data')

# 12.6 Private and public attribute

In [55]:
# File access1.py
traceMe = False
def trace(*args):
    if traceMe: print('[' + ' '.join(map(str, args)) + ']')
        
def Private(*privates): # privates in enclosing scope
    def onDecorator(aClass): # aClass in enclosing scope
        class onInstance: # wrapped in instance attribute
            def __init__(self, *args, **kargs):
                self.wrapped = aClass(*args, **kargs)
                
            def __getattr__(self, attr): # My attrs don't call getattr
                trace('get:', attr) # Others assumed in wrapped
                if attr in privates:
                    raise TypeError('private attribute fetch: ' + attr)
                else:
                    return getattr(self.wrapped, attr)
                
            def __setattr__(self, attr, value): # Outside accesses
                trace('set:', attr, value) # Others run normally
                if attr == 'wrapped': # Allow my attrs
                    self.__dict__[attr] = value # Avoid looping
                elif attr in privates:
                    raise TypeError('private attribute change: ' + attr)
                else:
                    setattr(self.wrapped, attr, value) # Wrapped obj attrs
        return onInstance # Or use __dict__
    return onDecorator

if __name__ == '__main__':
    traceMe = True
    
    @Private('data', 'size') # Doubler = Private(...)(Doubler)
    class Doubler:
        def __init__(self, label, start):
            self.label = label # Accesses inside the subject class
            self.data = start # Not intercepted: run normally
        def size(self):
            return len(self.data) # Methods run with no checking
        def double(self): # Because privacy not inherited
            for i in range(self.size()):
                self.data[i] = self.data[i] * 2
        def display(self):
            print('%s => %s' % (self.label, self.data))
            
    X = Doubler('X is', [1, 2, 3])
    Y = Doubler('Y is', [-10,-20,-30])
    print(X.label) # Accesses outside subject class
    X.display(); X.double(); X.display() # Intercepted: validated, delegated
    print(Y.label)
    Y.display(); Y.double()
    Y.label = 'Spam'
    Y.display()

[set: wrapped <__main__.Doubler object at 0x03671C70>]
[set: wrapped <__main__.Doubler object at 0x03671D50>]
[get: label]
X is
[get: display]
X is => [1, 2, 3]
[get: double]
[get: display]
X is => [2, 4, 6]
[get: label]
Y is
[get: display]
Y is => [-10, -20, -30]
[get: double]
[set: label Spam]
[get: display]
Spam => [-20, -40, -60]


In [63]:
# File access2.py
traceMe = False
def trace(*args):
    if traceMe: print('[' + ' '.join(map(str, args)) + ']')
        
def accessControl(failIf):
    def onDecorator(aClass):
        class onInstance:
            def __init__(self, *args, **kargs):
                self.__wrapped = aClass(*args, **kargs)
            def __getattr__(self, attr):
                trace('get:', attr)
                if failIf(attr):
                    raise TypeError('private attribute fetch: ' + attr)
                else:
                    return getattr(self.__wrapped, attr)
                
    def __setattr__(self, attr, value):
        trace('set:', attr, value)
        if attr == '_onInstance__wrapped':
            self.__dict__[attr] = value
        elif failIf(attr):
            raise TypeError('private attribute change: ' + attr)
        else:
            setattr(self.__wrapped, attr, value)
        return onInstance
    return onDecorator

def Private(*attributes):
    return accessControl(failIf=(lambda attr: attr in attributes))

def Public(*attributes):
    return accessControl(failIf=(lambda attr: attr not in attributes))

@Private('age') # Person = Private('age')(Person)
class Person: # Person = onInstance with state
    def __init__(self, name, age):
        self.name = name
        self.age = age


# Bài tập Excercise
## A. Quiz

**Câu 1:** Decorator class và function được sử dụng để làm gì?
## B. Coding

---
# <span style= 'color:blue'> Đáp án </span>

**1.** Decorator function thường được sử dụng để quản lý một hàm hoặc phương thức, hoặc thêm vào
nó là một lớp logic được chạy mỗi khi hàm hoặc phương thức được gọi. Người ta có thể
được sử dụng để ghi hoặc đếm các cuộc gọi đến một hàm, kiểm tra các loại đối số của nó, ...
Chúng cũng được sử dụng để "khai báo" các phương thức tĩnh (các hàm đơn giản trong một lớp
không được truyền một instance khi được gọi), cũng như các phương thức và thuộc tính của lớp. Lớp
decorator cũng tương tự, nhưng quản lý toàn bộ đối tượng và giao diện của chúng thay vì
gọi hàm.